In [ ]:
import os
# change the OS to use your project folder as the working directory
project_folder = "/content/drive/MyDrive/Colab_Notebooks/Aphasia/"
os.chdir(project_folder)

In [ ]:
def read_text_file(file_path):
 os.chdir(file_path)
 final_file=[]
 last_file=[]
 # iterate through all file
 for file in os.listdir(file_path):
    if os.path.isdir(file)==True:
      file_path_new=file_path+"/"+file
      for file in os.listdir(file_path_new):
        if file.startswith("."):
            continue
        if file.endswith(".xml"):
            document = f"{file_path_new}"+"/"+file
        # call read text file function
            with open(document, 'r', errors="ignore") as f:
               final_file.append(XMLreader_new(f))
    # Check whether file is in text format or not
    else:
        if file.startswith("."):
            continue
        if file.endswith(".xml"):
            document = f"{file_path}"+"/"+file
        # call read text file function
            with open(document, 'r',  errors="ignore") as f:
               final_file.append(XMLreader_new(f))
 return final_file

def XMLreader(filename=''):
  tree = parse(filename)
  root = tree.getroot()
  file=[]
  sentence=[]
  for elem in root:
      for gchild in elem:
                if gchild.text==None:
                    if sentence!=[]:
                      file.append(sentence)
                      sentence=[]
                      continue
                else:
                    sentence.append(gchild.text)
  return file

def XMLreader_comp(filename=''):
  corpus_relation=[]
  corpus_word=[]
  sentence_relation=[]
  sentence_word=[]
  from lxml import etree
  #xmldoc = minidom.parse(filename)
  #itemlist = xmldoc.getElementsByTagName('relation')
  #print(memoryElem.text)
  tree = parse(filename)
  root = tree.getroot()
  for elem in tree.findall('.//{http://www.talkbank.org/ns/talkbank}w'):
        print(elem.text)
        gra=elem.find('.//{http://www.talkbank.org/ns/talkbank}gra')
        if gra==None:
            continue
        elif gra.attrib['index'] == '1':
            corpus_word.append(sentence_word)
            corpus_relation.append(sentence_relation)
            sentence_word=[]
            sentence_relation=[]
            sentence_word.append(elem.text)
            sentence_relation.append(gra.attrib['relation'])
        else:
            sentence_word.append(elem.text)
            sentence_relation.append(gra.attrib['relation'])
  return corpus_word
def XMLreader_new(filename=''):
  corpus_relation=[]
  corpus_word=[]
  sentence_relation=[]
  sentence_word=[]
  from lxml import etree
  #xmldoc = minidom.parse(filename)
  #itemlist = xmldoc.getElementsByTagName('relation')
  #print(memoryElem.text)
  tree = parse(filename)
  root = tree.getroot()
  for u in root.findall(".//{http://www.talkbank.org/ns/talkbank}u"):
        sentence = []
        for w in u.findall(".//{http://www.talkbank.org/ns/talkbank}w"):
            sentence.append(w.text)
        corpus_word.append(sentence)
  return corpus_word

In [ ]:
import os
from xml.etree.ElementTree import parse
control_data=read_text_file("/content/drive/MyDrive/Colab_Notebooks/Aphasia/control")

In [ ]:
import os
from xml.etree.ElementTree import parse
aphasia_data=read_text_file("/content/drive/MyDrive/Colab_Notebooks/Aphasia/aphasia")

In [ ]:
aphasia_label=[1]*len(aphasia_data)
control_label=[0]*len(control_data)

In [ ]:
all_data=aphasia_data+control_data
all_label=aphasia_label+control_label

### **chatgpt 4o**

In [ ]:
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def detect_repetitive_phrases(participant_lists):
    for text_list in participant_lists:
        text_counter = Counter(text_list)
        if any(count > 1 for count in text_counter.values()):
            return 1  # Label as aphasia
    return 0  # Label as non-aphasia

def process_participants(data):
    labels = []
    for participant_lists in data:
        label = detect_repetitive_phrases(participant_lists)
        labels.append(label)
    return labels

# Assuming all_data and all_label are provided
# all_data: list of lists of lists (each participant's data)
# all_label: list of integers (each participant's true label)
# Example:
# all_data = [
#     [['www', 'www'], ['it', 'is', 'a', 'lovely', 'day', 'it', 'is', 'a', 'lovely', 'day'], ['the', 'sun', 'is', 'shining'], ['I', 'want', 'to', 'go', 'to', 'the', 'park', 'to', 'the', 'park']],
#     [['hello', 'hello'], ['today', 'is', 'a', 'good', 'day', 'today', 'is', 'a', 'good', 'day'], ['the', 'weather', 'is', 'nice'], ['let', 'us', 'go', 'to', 'the', 'beach', 'to', 'the', 'beach']],
#     # Add more participants here
# ]
# all_label = [1, 1, 0, ...]

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
data = all_data  # Use lists directly
labels = all_label  # Use lists directly

accuracies = []
f1_scores = []
specificities = []
sensitivities = []

for train_index, test_index in kf.split(data):
    train_data = [data[i] for i in train_index]
    test_data = [data[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Predict labels for test data
    predicted_labels = process_participants(test_data)

    accuracy = accuracy_score(test_labels, predicted_labels)
    f1 = f1_score(test_labels, predicted_labels)
    specificity = precision_score(test_labels, predicted_labels)
    sensitivity = recall_score(test_labels, predicted_labels)

    accuracies.append(accuracy)
    f1_scores.append(f1)
    specificities.append(specificity)
    sensitivities.append(sensitivity)

# Calculate standard deviations
accuracy_std = np.std(accuracies)
f1_std = np.std(f1_scores)
specificity_std = np.std(specificities)
sensitivity_std = np.std(sensitivities)

# Print cross-validation results
print(f"Cross-validation accuracies: {accuracies}")
print(f"Mean accuracy: {np.mean(accuracies)}, Std: {accuracy_std}")
print(f"F1 scores: {f1_scores}")
print(f"Mean F1 score: {np.mean(f1_scores)}, Std: {f1_std}")
print(f"Specificities: {specificities}")
print(f"Mean specificity: {np.mean(specificities)}, Std: {specificity_std}")
print(f"Sensitivities: {sensitivities}")
print(f"Mean sensitivity: {np.mean(sensitivities)}, Std: {sensitivity_std}")


Cross-validation accuracies: [0.677570093457944, 0.6635514018691588, 0.6854460093896714, 0.6713615023474179, 0.6525821596244131]
Mean accuracy: 0.670102233337721, Std: 0.011337129553777277
F1 scores: [0.807799442896936, 0.797752808988764, 0.8133704735376044, 0.8033707865168539, 0.7897727272727273]
Mean F1 score: 0.8024132478425772, Std: 0.00814237340085836
Specificities: [0.677570093457944, 0.6635514018691588, 0.6854460093896714, 0.6713615023474179, 0.6525821596244131]
Mean specificity: 0.670102233337721, Std: 0.011337129553777277
Sensitivities: [1.0, 1.0, 1.0, 1.0, 1.0]
Mean sensitivity: 1.0, Std: 0.0


In [ ]:
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def generate_rating(text_list):
    if text_list is None or len(text_list) == 0:
        return 0
    unique_words = set(text_list)
    if len(unique_words) == 1:
        return 1
    if len(unique_words) <= 3:
        return 2
    if all(word is not None and len(word) <= 3 for word in text_list):
        return 3
    if len(text_list) <= 5:
        return 4
    if len(text_list) <= 10:
        return 5
    if len(text_list) <= 15:
        return 6
    if len(text_list) <= 20:
        return 7
    if len(text_list) <= 25:
        return 8
    if len(text_list) <= 30:
        return 9
    return 10

def detect_repetitive_phrases(participant_lists):
    for text_list in participant_lists:
        if text_list is None:
            continue
        text_counter = Counter(text_list)
        if any(count > 1 for count in text_counter.values()):
            return 1  # Label as aphasia
    return 0  # Label as non-aphasia

def extract_features(participant_lists):
    num_repetitive_phrases = sum(detect_repetitive_phrases([text_list]) for text_list in participant_lists if text_list is not None)
    average_rating = np.mean([generate_rating(text_list) for text_list in participant_lists if text_list is not None])
    return [num_repetitive_phrases, average_rating]

# Assuming all_data and all_label are provided
# all_data: list of lists of lists (each participant's data)
# all_label: list of integers (each participant's true label)
# Example:
# all_data = [
#     [['www', 'www'], ['it', 'is', 'a', 'lovely', 'day', 'it', 'is', 'a', 'lovely', 'day'], ['the', 'sun', 'is', 'shining'], ['I', 'want', 'to', 'go', 'to', 'the', 'park', 'to', 'the', 'park']],
#     [['hello', 'hello'], ['today', 'is', 'a', 'good', 'day', 'today', 'is', 'a', 'good', 'day'], ['the', 'weather', 'is', 'nice'], ['let', 'us', 'go', 'to', 'the', 'beach', 'to', 'the', 'beach']],
#     # Add more participants here
# ]
# all_label = [1, 1, 0, ...]

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
data = all_data  # Use lists directly
labels = np.array(all_label)  # Convert to NumPy array for indexing

features = np.array([extract_features(participant_lists) for participant_lists in data])

accuracies = []
f1_scores = []
specificities = []
sensitivities = []

for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = LogisticRegression(random_state=1)
    model.fit(X_train, y_train)
    predicted_labels = model.predict(X_test)

    accuracy = accuracy_score(y_test, predicted_labels)
    f1 = f1_score(y_test, predicted_labels)
    specificity = precision_score(y_test, predicted_labels)
    sensitivity = recall_score(y_test, predicted_labels)

    accuracies.append(accuracy)
    f1_scores.append(f1)
    specificities.append(specificity)
    sensitivities.append(sensitivity)

# Calculate standard deviations
accuracy_std = np.std(accuracies)
f1_std = np.std(f1_scores)
specificity_std = np.std(specificities)
sensitivity_std = np.std(sensitivities)

# Print cross-validation results
print(f"Cross-validation accuracies: {accuracies}")
print(f"Mean accuracy: {np.mean(accuracies)}, Std: {accuracy_std}")
print(f"F1 scores: {f1_scores}")
print(f"Mean F1 score: {np.mean(f1_scores)}, Std: {f1_std}")
print(f"Specificities: {specificities}")
print(f"Mean specificity: {np.mean(specificities)}, Std: {specificity_std}")
print(f"Sensitivities: {sensitivities}")
print(f"Mean sensitivity: {np.mean(sensitivities)}, Std: {sensitivity_std}")

Cross-validation accuracies: [0.7429906542056075, 0.7663551401869159, 0.7605633802816901, 0.7323943661971831, 0.704225352112676]
Mean accuracy: 0.7413057785968145, Std: 0.022160937903184418
F1 scores: [0.8148148148148148, 0.8287671232876712, 0.8294314381270903, 0.8093645484949832, 0.7864406779661017]
Mean F1 score: 0.8137637205381323, Std: 0.015732677062400474
Specificities: [0.7960526315789473, 0.8066666666666666, 0.8104575163398693, 0.7756410256410257, 0.7435897435897436]
Mean specificity: 0.7864815167632505, Std: 0.02462159834322401
Sensitivities: [0.8344827586206897, 0.852112676056338, 0.8493150684931506, 0.8461538461538461, 0.8345323741007195]
Mean sensitivity: 0.8433193446849488, Std: 0.007437766929497693


WAB-AQ
Western Aphasia Battery-Aphasia Quotient 中的fluency


### **chatGPT 4**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

def create_model():
    # Use MaxAbsScaler which is recommended for sparse data and change solver and increase max_iter
    pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(solver='saga', max_iter=1000))
    return pipeline

# Define classifier pipeline
classifier = create_model()

# Setup 5-fold cross-validation
kf = StratifiedKFold(n_splits=5)
f1_scores = []
sensitivities = []
specificities = []

for train_index, test_index in kf.split(features, all_label):
    train_data, test_data = features[train_index], features[test_index]
    train_labels, test_labels = np.array(all_label)[train_index], np.array(all_label)[test_index]

    classifier.fit(train_data, train_labels)
    predictions = classifier.predict(test_data)

    # Calculate metrics
    accuracy = accuracy_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    sensitivity = recall_score(test_labels, predictions)
    tn, fp, fn, tp = confusion_matrix(test_labels, predictions).ravel()
    specificity = tn / (tn+fp) if (tn+fp) > 0 else 0

    accuracies.append(accuracy)
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate mean and standard deviation of the metrics
acc_mean, acc_std = np.mean(accuracies), np.std(accuracies)
f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
sensitivity_mean, sensitivity_std = np.mean(sensitivities), np.std(sensitivities)
specificity_mean, specificity_std = np.mean(specificities), np.std(specificities)

print("Accuracy: Mean = {:.2f}, Std = {:.2f}".format(acc_mean, acc_std))
print("F1 Score: Mean = {:.2f}, Std = {:.2f}".format(f1_mean, f1_std))
print("Sensitivity: Mean = {:.2f}, Std = {:.2f}".format(sensitivity_mean, sensitivity_std))
print("Specificity: Mean = {:.2f}, Std = {:.2f}".format(specificity_mean, specificity_std))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy: Mean = 0.67, Std = 0.16
F1 Score: Mean = 0.74, Std = 0.17
Sensitivity: Mean = 0.80, Std = 0.25
Specificity: Mean = 0.41, Std = 0.30


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

# Function to preprocess and flatten text data, handling None values
def preprocess_text_data(all_data):
    return [" ".join([" ".join(filter(None, sentence)) for sentence in participant]) for participant in all_data]

# Basic feature extraction using TF-IDF combined with simple linguistic features
def extract_features(text_data):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_features = tfidf_vectorizer.fit_transform(text_data)

    # Calculate simple linguistic features
    additional_features = []
    for text in text_data:
        words = text.split()
        sentence_lengths = [len(sentence.split()) for sentence in text.split('.')]
        avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
        type_token_ratio = len(set(words)) / len(words) if words else 0
        additional_features.append([avg_sentence_length, type_token_ratio])

    additional_features = np.array(additional_features)
    return np.hstack((tfidf_features.toarray(), additional_features))

# Create a logistic regression model
def create_model():
    return make_pipeline(MaxAbsScaler(), LogisticRegression(solver='saga', max_iter=1000))

processed_data = preprocess_text_data(all_data)
features = extract_features(processed_data)

classifier = create_model()

# Setup 5-fold cross-validation
kf = StratifiedKFold(n_splits=5)
accuracies=[]
f1_scores = []
sensitivities = []
specificities = []

for train_index, test_index in kf.split(features, all_label):
    train_features, test_features = features[train_index], features[test_index]
    train_labels, test_labels = np.array(all_label)[train_index], np.array(all_label)[test_index]

    classifier.fit(train_features, train_labels)
    predictions = classifier.predict(test_features)

    accuracy = accuracy_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    sensitivity = recall_score(test_labels, predictions)
    tn, fp, fn, tp = confusion_matrix(test_labels, predictions).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    accuracies.append(accuracy)
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate mean and standard deviation of the metrics
acc_mean, acc_std = np.mean(accuracies), np.std(accuracies)
f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
sensitivity_mean, sensitivity_std = np.mean(sensitivities), np.std(sensitivities)
specificity_mean, specificity_std = np.mean(specificities), np.std(specificities)

print("Accuracy: Mean = {:.2f}, Std = {:.2f}".format(acc_mean, acc_std))
print("F1 Score: Mean = {:.2f}, Std = {:.2f}".format(f1_mean, f1_std))
print("Sensitivity: Mean = {:.2f}, Std = {:.2f}".format(sensitivity_mean, sensitivity_std))
print("Specificity: Mean = {:.2f}, Std = {:.2f}".format(specificity_mean, specificity_std))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy: Mean = 0.67, Std = 0.16
F1 Score: Mean = 0.74, Std = 0.17
Sensitivity: Mean = 0.80, Std = 0.25
Specificity: Mean = 0.41, Std = 0.30


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, precision_score

# Example feature calculation: average sentence length and type-token ratio (lexical diversity)
def calculate_features(text):
    sentences = text.split('.')
    avg_sentence_length = np.mean([len(sentence.split()) for sentence in sentences if sentence.strip() != ''])
    words = text.split()
    lexical_diversity = len(set(words)) / len(words) if words else 0
    return avg_sentence_length, lexical_diversity

# Function to preprocess and flatten text data, handling None values
def preprocess_text_data(all_data):
    return [" ".join([" ".join(filter(None, sentence)) for sentence in participant]) for participant in all_data]

# Threshold-based prediction for aphasia
def predict_aphasia(features, sentence_length_threshold, diversity_threshold):
    avg_sentence_length, lexical_diversity = features
    if avg_sentence_length < sentence_length_threshold or lexical_diversity < diversity_threshold:
        return 1  # Indicates aphasia
    else:
        return 0  # No aphasia

# Perform 5-fold cross-validation to calculate accuracy, F1 score, sensitivity, and specificity
def perform_cross_validation(features, labels):
    kf = StratifiedKFold(n_splits=5)
    accuracy_scores = []
    f1_scores = []
    sensitivity_scores = []
    specificity_scores = []

    for train_index, test_index in kf.split(features, labels):
        test_features = [features[i] for i in test_index]
        test_labels = [labels[i] for i in test_index]

        predictions = [predict_aphasia(f, 5, 0.5) for f in test_features]  # Using example thresholds

        accuracy_scores.append(accuracy_score(test_labels, predictions))
        f1_scores.append(f1_score(test_labels, predictions))
        sensitivity_scores.append(recall_score(test_labels, predictions))
        tn, fp, fn, tp = confusion_matrix(test_labels, predictions).ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificity_scores.append(specificity)

    # Calculate mean and standard deviation of the metrics
    print_evaluation(accuracy_scores, f1_scores, sensitivity_scores, specificity_scores)

# Print evaluation metrics
def print_evaluation(accuracy_scores, f1_scores, sensitivity_scores, specificity_scores):
    print("Accuracy: Mean = {:.2f}, Std = {:.2f}".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
    print("F1 Score: Mean = {:.2f}, Std = {:.2f}".format(np.mean(f1_scores), np.std(f1_scores)))
    print("Sensitivity: Mean = {:.2f}, Std = {:.2f}".format(np.mean(sensitivity_scores), np.std(sensitivity_scores)))
    print("Specificity: Mean = {:.2f}, Std = {:.2f}".format(np.mean(specificity_scores), np.std(specificity_scores)))

processed_data = preprocess_text_data(all_data)
features = [calculate_features(text) for text in processed_data]

perform_cross_validation(features, all_label)



Accuracy: Mean = 0.67, Std = 0.00
F1 Score: Mean = 0.80, Std = 0.00
Sensitivity: Mean = 1.00, Std = 0.00
Specificity: Mean = 0.00, Std = 0.00


### **claude3**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix

def extract_features(participant_utterances):
    features = {
        'repeated_words_ratio': 0,
        'repeated_phrases_ratio': 0,
        'short_sentences_ratio': 0,
        'nonsense_words_ratio': 0
    }
    total_words = 0

    for utterance in participant_utterances:
        # Filter out None values
        utterance = [word for word in utterance if word is not None]
        total_words += len(utterance)
        repeated_words = len([word for word in set(utterance) if utterance.count(word) > 1])
        repeated_phrases = len(detect_repeated_phrases(utterance))
        short_sentence = 1 if len(utterance) <= 3 else 0
        nonsense_words = len([word for word in utterance if len(str(word)) > 1 and str(word).lower() not in basic_english_words])

        features['repeated_words_ratio'] += repeated_words
        features['repeated_phrases_ratio'] += repeated_phrases
        features['short_sentences_ratio'] += short_sentence
        features['nonsense_words_ratio'] += nonsense_words

    for key in features:
        features[key] /= max(total_words, 1)

    return list(features.values())

def classify_aphasia(features, threshold=0.1):
    return int(any(feature > threshold for feature in features))

def detect_repeated_phrases(word_list):
    phrases = []
    word_list = [str(word) for word in word_list if word is not None]  # Convert to string and filter None
    for i in range(len(word_list)):
        for j in range(i+2, len(word_list)):
            phrase = ' '.join(word_list[i:j])
            if word_list.count(phrase) > 1:
                phrases.append(phrase)
    return list(set(phrases))

def cross_validate(X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1_scores = []
    specificities = []
    sensitivities = []
    accuracies=[]

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train (in this case, just set the threshold)
        threshold = np.mean(X_train.max(axis=1))

        # Predict
        y_pred = [classify_aphasia(features, threshold) for features in X_test]

        # Calculate metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
        specificities.append(tn / (tn + fp))
        sensitivities.append(tp / (tp + fn))

    return {
        'accuracy':(np.mean(accuracies), np.std(accuracies)),
        'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
        'specificity': (np.mean(specificities), np.std(specificities)),
        'sensitivity': (np.mean(sensitivities), np.std(sensitivities))
    }

# Basic set of common English words (you should expand this)
basic_english_words = set(['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'it',
                           'for', 'not', 'on', 'with', 'he', 'as', 'you', 'do', 'at', 'this',
                           'but', 'his', 'by', 'from', 'they', 'we', 'say', 'her', 'she', 'or',
                           'an', 'will', 'my', 'one', 'all', 'would', 'there', 'their', 'what',
                           'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go', 'me',
                           'is', 'day', 'lovely', 'www'])

# Use your data
# all_data = # Your all_data variable here
# all_label = # Your all_label variable here

# Extract features for each participant
X = np.array([extract_features(participant) for participant in all_data])
y = np.array(all_label)

# Perform cross-validation
results = cross_validate(X, y)

# Print results
for metric, (mean, std) in results.items():
    print(f"{metric}: {mean:.3f} ± {std:.3f}")

accuracy: 0.605 ± 0.034
f1_score: 0.623 ± 0.036
specificity: 0.844 ± 0.037
sensitivity: 0.488 ± 0.033


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
import random

def generate_rating(utterance):
    # This is a simplified rating generation based on the scales you provided
    # You may need to refine this based on more specific criteria
    if not utterance or all(len(word) < 2 for word in utterance):
        return 0
    elif len(utterance) < 3:
        return random.choice([1, 2])
    elif len(utterance) < 5:
        return random.choice([3, 4, 5])
    elif len(utterance) < 8:
        return random.choice([6, 7])
    else:
        return random.choice([8, 9, 10])

def extract_features(participant_utterances):
    features = {
        'repeated_words_ratio': 0,
        'repeated_phrases_ratio': 0,
        'short_sentences_ratio': 0,
        'nonsense_words_ratio': 0,
        'average_rating': 0
    }
    total_words = 0
    total_utterances = len(participant_utterances)

    for utterance in participant_utterances:
        utterance = [word for word in utterance if word is not None]
        total_words += len(utterance)
        repeated_words = len([word for word in set(utterance) if utterance.count(word) > 1])
        repeated_phrases = len(detect_repeated_phrases(utterance))
        short_sentence = 1 if len(utterance) <= 3 else 0
        nonsense_words = len([word for word in utterance if len(str(word)) > 1 and str(word).lower() not in basic_english_words])

        features['repeated_words_ratio'] += repeated_words
        features['repeated_phrases_ratio'] += repeated_phrases
        features['short_sentences_ratio'] += short_sentence
        features['nonsense_words_ratio'] += nonsense_words
        features['average_rating'] += generate_rating(utterance)

    for key in features:
        if key == 'average_rating':
            features[key] /= total_utterances
        else:
            features[key] /= max(total_words, 1)

    return list(features.values())

def classify_aphasia(features, threshold=0.1):
    # Adjust the classification based on the new features
    linguistic_features = features[:4]  # First 4 features are the original ones
    rating = features[4]  # The 5th feature is the average rating

    if any(feature > threshold for feature in linguistic_features) or rating < 5:
        return 1  # Classified as having aphasia
    return 0  # Classified as not having aphasia

def detect_repeated_phrases(word_list):
    phrases = []
    word_list = [str(word) for word in word_list if word is not None]
    for i in range(len(word_list)):
        for j in range(i+2, len(word_list)):
            phrase = ' '.join(word_list[i:j])
            if word_list.count(phrase) > 1:
                phrases.append(phrase)
    return list(set(phrases))

def cross_validate(X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1_scores = []
    specificities = []
    sensitivities = []
    accuracies=[]

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train (in this case, just set the threshold)
        threshold = np.mean(X_train[:, :4].max(axis=1))  # Only use linguistic features for threshold

        # Predict
        y_pred = [classify_aphasia(features, threshold) for features in X_test]

        # Calculate metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        accuracies.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
        specificities.append(tn / (tn + fp))
        sensitivities.append(tp / (tp + fn))

    return {
        'accuracy':(np.mean(accuracies), np.std(accuracies)),
        'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
        'specificity': (np.mean(specificities), np.std(specificities)),
        'sensitivity': (np.mean(sensitivities), np.std(sensitivities))
    }

# Basic set of common English words (you should expand this)
basic_english_words = set(['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'it',
                           'for', 'not', 'on', 'with', 'he', 'as', 'you', 'do', 'at', 'this',
                           'but', 'his', 'by', 'from', 'they', 'we', 'say', 'her', 'she', 'or',
                           'an', 'will', 'my', 'one', 'all', 'would', 'there', 'their', 'what',
                           'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go', 'me',
                           'is', 'day', 'lovely', 'www'])

# Use your data
# all_data = # Your all_data variable here
# all_label = # Your all_label variable here

# Extract features for each participant
X = np.array([extract_features(participant) for participant in all_data])
y = np.array(all_label)

# Perform cross-validation
results = cross_validate(X, y)

# Print results
for metric, (mean, std) in results.items():
    print(f"{metric}: {mean:.3f} ± {std:.3f}")

accuracy: 0.608 ± 0.036
f1_score: 0.627 ± 0.039
specificity: 0.844 ± 0.037
sensitivity: 0.492 ± 0.036
